In [30]:
import os.path as op
import glob
import pandas as pd
import numpy as np
from mubench.utils.macro import CLASSIFICATION_DATASET, REGRESSION_DATASET
from mubench.utils.io import init_dir


In [23]:
CLASSIFICATION_METRICS = ['roc-auc', 'prc-auc', 'ece', 'mce', 'nll', 'brier']
REGRESSION_METRICS = ['rmse', 'mae', 'nll', 'ce']
LARGER_BETTER_LOOKUP = {
    'roc-auc': True,
    'prc-auc': True,
    'ece': False,
    'mce': False,
    'nll': False,
    'brier': False,
    'rmse': False,
    'mae': False,
    'ce': False
}

In [24]:
def get_ranks(values, smaller_is_better=True):
    arr = np.array(values)
    if smaller_is_better:
        order = arr.argsort()
    else:
        order = arr.argsort()[::-1]
    rnks = order.argsort()
    return rnks + 1


In [25]:
result_path = '../output/RESULTS/'
score_path = op.join(result_path, 'scores')
result_files = list(glob.glob(op.join(score_path, '*.csv')))

In [26]:
cla_dt_md_unc_mtr = dict()
reg_dt_md_unc_mtr = dict()
for result_file in result_files:

    file_name = op.basename(result_file)
    model_name = '-'.join(file_name.split('-')[:-1])
    dataset_name = file_name.split('-')[-1].split('.')[0]

    if dataset_name in CLASSIFICATION_DATASET:
        dataset_model_uncertainty_metric = cla_dt_md_unc_mtr
    elif dataset_name in REGRESSION_DATASET:
        dataset_model_uncertainty_metric = reg_dt_md_unc_mtr
    else:
        raise ValueError

    if dataset_name not in dataset_model_uncertainty_metric.keys():
        dataset_model_uncertainty_metric[dataset_name] = dict()

    df = pd.read_csv(result_file)

    uncertainty_methods = df.method
    uncertainty_methods = [um.split('-')[-1] for um in uncertainty_methods]

    metric_means = [k for k in df.keys() if k.endswith('-mean')]
    metric_names = [m[:-5] for m in metric_means]
    column_headers = ['method'] + metric_means

    uncertainty_metric = dict()
    for item in df[column_headers].to_numpy():
        uncertainty_method = item[0].split('-')[-1]
        metric_values = item[1:]
        assert len(metric_values) == len(metric_names)

        uncertainty_metric[uncertainty_method] = {n: v for n, v in zip(metric_names, metric_values)}

    dataset_model_uncertainty_metric[dataset_name][model_name] = uncertainty_metric

cla_fl_dt_md_unc_mtr = pd.json_normalize(cla_dt_md_unc_mtr, sep='_').to_dict()
reg_fl_dt_md_unc_mtr = pd.json_normalize(reg_dt_md_unc_mtr, sep='_').to_dict()

cla_dt_fl_md_unc_mtr = {dt: pd.json_normalize(cla_md_unc_mtr, sep='_').to_dict() for dt, cla_md_unc_mtr in cla_dt_md_unc_mtr.items()}
reg_dt_fl_md_unc_mtr = {dt: pd.json_normalize(reg_md_unc_mtr, sep='_').to_dict() for dt, reg_md_unc_mtr in reg_dt_md_unc_mtr.items()}



In [27]:
classification_datasets = list(cla_dt_fl_md_unc_mtr.keys())
regression_datasets = list(reg_dt_fl_md_unc_mtr.keys())

cla_metric_ranks = {dt: {mtr: dict() for mtr in CLASSIFICATION_METRICS} for dt in classification_datasets}
reg_metric_ranks = {dt: {mtr: dict() for mtr in REGRESSION_METRICS} for dt in regression_datasets}

for dt, mtr_vals in cla_metric_ranks.items():
    for mtr in mtr_vals:
        md_unc_scores = {'_'.join(k.split('_')[1:-1]): val[0] for k, val in cla_fl_dt_md_unc_mtr.items() if k.startswith(dt) and k.endswith(mtr)}
        ranks = get_ranks(list(md_unc_scores.values()), smaller_is_better=not LARGER_BETTER_LOOKUP[mtr])
        md_unc_ranks = {k: r for k, r in zip(md_unc_scores, ranks)}
        mtr_vals[mtr] = md_unc_ranks

for dt, mtr_vals in reg_metric_ranks.items():
    for mtr in mtr_vals:
        md_unc_scores = {'_'.join(k.split('_')[1:-1]): val[0] for k, val in reg_fl_dt_md_unc_mtr.items() if k.startswith(dt) and k.endswith(mtr)}
        ranks = get_ranks(list(md_unc_scores.values()), smaller_is_better=not LARGER_BETTER_LOOKUP[mtr])
        md_unc_ranks = {k: r for k, r in zip(md_unc_scores, ranks)}
        mtr_vals[mtr] = md_unc_ranks


In [31]:
output_dir = op.join(result_path, 'ranks')
init_dir(output_dir)

for dataset in classification_datasets:
    dataset_metric_ranks = cla_metric_ranks[dataset]
    df = pd.DataFrame(dataset_metric_ranks)
    df.to_csv(op.join(output_dir, f'{dataset}.csv'))

for dataset in regression_datasets:
    dataset_metric_ranks = reg_metric_ranks[dataset]
    df = pd.DataFrame(dataset_metric_ranks)
    df.to_csv(op.join(output_dir, f'{dataset}.csv'))
